# Shortest-Path Problem datasets

## 1. Import

In [89]:
import numpy as np
import networkx as nx
import itertools as iter
from scipy.spatial.distance import cdist

from torch_geometric.utils.convert import from_networkx

import random
import pickle

## 2. Utils

In [90]:
def draw(graph, ax):
    """
    Draws the graph as a matplotlib plot.
    Depots are colored in red. Edges that have been
    traveresed
    """

    # draw nodes according to color and position attribute
    pos = nx.get_node_attributes(graph, "coordinates")
    nx.draw_networkx_nodes(
        graph, pos, ax=ax, node_size=100
    )
    labels_pos = {k: (v + np.array([0, 0.03])) for k, v in pos.items()}
    nx.draw_networkx_labels(
        graph, labels_pos, ax=ax
    )

    # draw edges
    nx.draw_networkx_edges(graph, pos)

## 3. Generating Distance Matrices

In [91]:
def build_graph(num_nodes):
    G = nx.watts_strogatz_graph(num_nodes, 4, 0.1)
    node_position = {
        i: coordinates for i, coordinates in enumerate(np.random.rand(num_nodes, 2))
    }
    nx.set_node_attributes(G, node_position, "coordinates")

    positions = np.array([node_position[index] for index in range(num_nodes)])
    distances = cdist(positions, positions)

    edges = []
    for (u,v,w) in G.edges(data=True):
        w['dist'] = distances[u,v]

    # add distance as edge attribute
    edge_distance_feature = {(u,v):distances[u,v] for u in range(num_nodes) for v in range(num_nodes) if u != v}
    nx.set_edge_attributes(G, edge_distance_feature, 'dist')
    return G, distances

G, distances = build_graph(50)
print(G.number_of_edges())

100


## 4. Generate dataset

In [92]:
def get_route_length(route, dist_mat):
    """
    Returns the length of the routes
    """
    length = 0
    for i in range(len(route)-1):
        length += dist_mat[route[i], route[i+1]]
    return length

def build_dataset(size, num_nodes):
    graphs = []
    targets = []
    opts = []
    for i in range(size):
        if i % 10000 == 0:
            print(f'Building graph {i}/{size}')
        graph, dist_mat = build_graph(num_nodes)
        graph_pyg = from_networkx(graph, group_node_attrs=['coordinates'], group_edge_attrs=['dist'])
        pair_nodes = list(iter.combinations(range(num_nodes), 2))[random.randint(0, (num_nodes*(num_nodes-1)/2)-1)]
        opt_routes = nx.shortest_path(graph, weight='dist', source=pair_nodes[0], target=pair_nodes[1])
        opt_length = get_route_length(opt_routes, dist_mat)


        # Assign optimum length
        graph_pyg.y = opt_length

        # Saving
        graphs.append(graph_pyg)
        targets.append(opt_routes)
        opts.append(opt_length)
    return graphs, targets, opts

In [93]:
data_20_train = build_dataset(50000,20)
with open('PATH_dataset_20_train.pkl', 'wb') as file:
    pickle.dump(data_20_train, file)

Building graph 0/50000
Building graph 10000/50000
Building graph 20000/50000
Building graph 30000/50000
Building graph 40000/50000


In [94]:
data_20_test = build_dataset(100,20)
with open('PATH_dataset_20_test.pkl', 'wb') as file:
    pickle.dump(data_20_test, file)

Building graph 0/100


In [95]:
data_50_train = build_dataset(10000,50)
with open('PATH_dataset_50_train.pkl', 'wb') as file:
    pickle.dump(data_50_train, file)

Building graph 0/10000


In [96]:
 data_50_test = build_dataset(100,50)
with open('PATH_dataset_50_test.pkl', 'wb') as file:
    pickle.dump(data_50_test, file)

Building graph 0/100
